# This notebook is meant for testing out tensorboard

In [1]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [4]:
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [5]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

In [7]:
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()
scaled_housing_data_plus_bias = stdscaler.fit_transform(housing_data_plus_bias)

In [8]:
n_epochs = 1000
learning_rate = 0.01

# Gradient Descent

In [9]:
tf.reset_default_graph()
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, -1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  14.063578
Epoch 100 MSE =  5.019601
Epoch 200 MSE =  4.8394227
Epoch 300 MSE =  4.824713
Epoch 400 MSE =  4.820083
Epoch 500 MSE =  4.8168607
Epoch 600 MSE =  4.8143153
Epoch 700 MSE =  4.812273
Epoch 800 MSE =  4.8106265
Epoch 900 MSE =  4.809294


# Mini-batch Gradient Descent

In [11]:
from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}'.format(root_logdir, now)

In [12]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, -1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [13]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [14]:
batch_size = int(m/2)
n_batches = int(np.ceil(m / batch_size))

In [15]:
def fetch_batch(epoch, batch_index, batch_size):
    start = batch_index * batch_size
    end = start + batch_size
    return scaled_housing_data_plus_bias[start:end], housing.target.reshape(-1, 1)[start:end]

In [16]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

In [17]:
file_writer.close()